# Capstone Mini-Project: Data Wrangling

### Daniel Lee
### June 27, 2018

For the data wrangling portion, I divided my work into seven steps. For the first six steps, I am reading in and cleaning six different datasets from 2016 county level data found from the web. In the final step, I combine the six datasets together:

1. prescription_rates: opioid prescription rates
2. education
3. population
4. poverty
5. unemployment
6. death rates from drug overdose
7. merge data together on county code (FIPS code)

In [282]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## 1. Opioid Prescription Rates

In [283]:
# scrape prescription data from internet using BeautifulSoup
url = 'https://www.cdc.gov/drugoverdose/maps/rxcounty2016.html'
r = requests.get(url)
r.status_code
soup = BeautifulSoup(r.content, "html.parser")
stat_table = soup.find_all('table', class_ = 'table table-striped')
stat_table = stat_table[0]

# write the table into a txt with context manager
with open ('prescribing_stats.txt', 'w') as r:
    for row in stat_table.find_all('tr'): # find all rows in the table
        for cell in row.find_all('td'):
            r.write(cell.text.ljust(30))
        r.write('\n')

In [284]:
# read in the prescription data into a pandas df
prescription_rates = pd.read_csv('prescribing_stats.txt', delimiter = r"[ \t]{2,}", header = None, engine = 'python', na_values = '–')
prescription_rates.columns = ['county', 'state', 'county_code', 'prescribe_rate']

In [285]:
prescription_rates.head()

,county,state,county_code,prescribe_rate
0,"Aleutians East, AK",AK,2013,NaN
1,"Aleutians West, AK",AK,2016,NaN
2,"Anchorage, AK",AK,2020,66.3
3,"Bethel, AK",AK,2050,NaN
4,"Bristol Bay, AK",AK,2060,NaN


In [286]:
prescription_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 4 columns):
county            3143 non-null object
state             3143 non-null object
county_code       3143 non-null int64
prescribe_rate    2962 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 98.3+ KB


In [287]:
# remove null prescription_rates values from prescription_rates (2962/3143 non-null values)
prescription_rates_no_na = prescription_rates.loc[prescription_rates.prescribe_rate.notnull(), :]

In [288]:
prescription_rates_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2962 entries, 2 to 3142
Data columns (total 4 columns):
county            2962 non-null object
state             2962 non-null object
county_code       2962 non-null int64
prescribe_rate    2962 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 115.7+ KB


## 2. Education

In [289]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/Education.xls?v=43125

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1529k  100 1529k    0     0   526k      0  0:00:02  0:00:02 --:--:--  526k


In [290]:
# change name of downloaded file using bash command mv
!mv Education.xls?v=43125 education.xls

In [291]:
# load education data from Excel file
xl = pd.ExcelFile('education.xls')
xl.sheet_names

['Education 1970 to 2016']

In [292]:
# load a sheet into a pandas df by index
education = xl.parse(0, skiprows = 4)

# select columns that only contain relevant data for 2016
education = education[list(education.columns[:3]) + [column for column in education.columns if '2016' in column]]

In [293]:
education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3283 entries, 0 to 3282
Data columns (total 11 columns):
FIPS Code                                                                     3283 non-null int64
State                                                                         3283 non-null object
Area name                                                                     3283 non-null object
Less than a high school diploma, 2012-2016                                    3273 non-null float64
High school diploma only, 2012-2016                                           3273 non-null float64
Some college or associate's degree, 2012-2016                                 3273 non-null float64
Bachelor's degree or higher, 2012-2016                                        3273 non-null float64
Percent of adults with less than a high school diploma, 2012-2016             3273 non-null float64
Percent of adults with a high school diploma only, 2012-2016                  3273 non-null float

In [294]:
# drop all na values (3273/3283 non null values)
education_no_na = education.dropna(how = 'any')

In [295]:
education_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3273 entries, 0 to 3282
Data columns (total 11 columns):
FIPS Code                                                                     3273 non-null int64
State                                                                         3273 non-null object
Area name                                                                     3273 non-null object
Less than a high school diploma, 2012-2016                                    3273 non-null float64
High school diploma only, 2012-2016                                           3273 non-null float64
Some college or associate's degree, 2012-2016                                 3273 non-null float64
Bachelor's degree or higher, 2012-2016                                        3273 non-null float64
Percent of adults with less than a high school diploma, 2012-2016             3273 non-null float64
Percent of adults with a high school diploma only, 2012-2016                  3273 non-null float

## 3. Population

In [296]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/PopulationEstimates.xls?v=43209

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4382k  100 4382k    0     0  1586k      0  0:00:02  0:00:02 --:--:-- 1586k


In [297]:
# rename the downloaded file to education.xls using bash command mv
!mv PopulationEstimates.xls?v=43209 population.xls

In [298]:
# load population information from Excel file
xl = pd.ExcelFile('population.xls')
xl.sheet_names

['Population Estimates 2010-17', 'Variable Descriptions']

In [299]:
# load a sheet into a pandas df by index 
population = xl.parse(0, skiprows = 2)

# select columns that only contain relevant data for 2016
population = population[list(population.columns[:3]) + [column for column in population.columns if '2016' in column]]

In [300]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3273 entries, 0 to 3272
Data columns (total 19 columns):
FIPS                        3273 non-null int64
State                       3273 non-null object
Area_Name                   3273 non-null object
POP_ESTIMATE_2016           3273 non-null int64
N_POP_CHG_2016              3194 non-null float64
Births_2016                 3194 non-null float64
Deaths_2016                 3194 non-null float64
NATURAL_INC_2016            3194 non-null float64
INTERNATIONAL_MIG_2016      3194 non-null float64
DOMESTIC_MIG_2016           3194 non-null float64
NET_MIG_2016                3194 non-null float64
RESIDUAL_2016               3194 non-null float64
GQ_ESTIMATES_2016           3193 non-null float64
R_birth_2016                3194 non-null float64
R_death_2016                3194 non-null float64
R_NATURAL_INC_2016          3194 non-null float64
R_INTERNATIONAL_MIG_2016    3194 non-null float64
R_DOMESTIC_MIG_2016         3194 non-null float64

In [301]:
# drop all na values (3273/3283 non null values)
pop_2016_no_na = population.dropna(how = 'any')

In [302]:
pop_2016_no_na.head()

,FIPS,State,Area_Name,POP_ESTIMATE_2016,N_POP_CHG_2016,Births_2016,Deaths_2016,NATURAL_INC_2016,INTERNATIONAL_MIG_2016,DOMESTIC_MIG_2016,NET_MIG_2016,RESIDUAL_2016,GQ_ESTIMATES_2016,R_birth_2016,R_death_2016,R_NATURAL_INC_2016,R_INTERNATIONAL_MIG_2016,R_DOMESTIC_MIG_2016,R_NET_MIG_2016
1,1000,AL,Alabama,4860545,9687.0,59095.0,52077.0,7018.0,4678.0,-1964.0,2714.0,-45.0,118989.0,12.170229,10.724918,1.445311,0.963404,-0.404473,0.558931
2,1001,AL,Autauga County,55278,440.0,666.0,505.0,161.0,15.0,266.0,281.0,-2.0,455.0,12.096335,9.172146,2.924189,0.272440,4.831269,5.103709
3,1003,AL,Baldwin County,207509,4646.0,2312.0,2045.0,267.0,169.0,4205.0,4374.0,5.0,2275.0,11.267825,9.966567,1.301258,0.823643,20.493601,21.317244
4,1005,AL,Barbour County,25774,-490.0,272.0,269.0,3.0,-5.0,-488.0,-493.0,0.0,2977.0,10.453899,10.338599,0.115300,-0.192167,-18.755525,-18.947692
5,1007,AL,Bibb County,22633,72.0,290.0,207.0,83.0,20.0,-32.0,-12.0,1.0,2201.0,12.833562,9.160508,3.673054,0.885073,-1.416117,-0.531044


## 4. Poverty Estimates

In [303]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.xls?v=43125

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  876k  100  876k    0     0   324k      0  0:00:02  0:00:02 --:--:--  324k


In [304]:
# rename the downloaded file to education.xls
!mv PovertyEstimates.xls?v=43125 poverty.xls

In [305]:
# load poverty estimate data from Excel file
xl = pd.ExcelFile('poverty.xls')
xl.sheet_names

['Poverty Data 2016', 'Variable Descriptions']

In [306]:
# load a sheet into a pandas df by index 
poverty = xl.parse(0, skiprows = 3)

In [307]:
# select columns that only contain relevant data for 2016
poverty = poverty[list(poverty.columns[:3]) + [column for column in poverty.columns if '2016' in column]]

In [308]:
poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194 entries, 0 to 3193
Data columns (total 30 columns):
FIPStxt            3194 non-null int64
State              3194 non-null object
Area_Name          3194 non-null object
POVALL_2016        3193 non-null float64
CI90LBAll_2016     3193 non-null float64
CI90UBALL_2016     3193 non-null float64
PCTPOVALL_2016     3193 non-null float64
CI90LBALLP_2016    3193 non-null float64
CI90UBALLP_2016    3193 non-null float64
POV017_2016        3193 non-null float64
CI90LB017_2016     3193 non-null float64
CI90UB017_2016     3193 non-null float64
PCTPOV017_2016     3193 non-null float64
CI90LB017P_2016    3193 non-null float64
CI90UB017P_2016    3193 non-null float64
POV517_2016        3193 non-null float64
CI90LB517_2016     3193 non-null float64
CI90UB517_2016     3193 non-null float64
PCTPOV517_2016     3193 non-null float64
CI90LB517P_2016    3193 non-null float64
CI90UB517P_2016    3193 non-null float64
MEDHHINC_2016      3193 non-null flo

In [309]:
# remove columns with only 52 entries and drop rows with na values (3193/3194 non null values)
poverty_no_na = poverty[list(poverty.columns[:-7])].dropna(how = 'any')

In [310]:
poverty_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3193
Data columns (total 23 columns):
FIPStxt            3193 non-null int64
State              3193 non-null object
Area_Name          3193 non-null object
POVALL_2016        3193 non-null float64
CI90LBAll_2016     3193 non-null float64
CI90UBALL_2016     3193 non-null float64
PCTPOVALL_2016     3193 non-null float64
CI90LBALLP_2016    3193 non-null float64
CI90UBALLP_2016    3193 non-null float64
POV017_2016        3193 non-null float64
CI90LB017_2016     3193 non-null float64
CI90UB017_2016     3193 non-null float64
PCTPOV017_2016     3193 non-null float64
CI90LB017P_2016    3193 non-null float64
CI90UB017P_2016    3193 non-null float64
POV517_2016        3193 non-null float64
CI90LB517_2016     3193 non-null float64
CI90UB517_2016     3193 non-null float64
PCTPOV517_2016     3193 non-null float64
CI90LB517P_2016    3193 non-null float64
CI90UB517P_2016    3193 non-null float64
MEDHHINC_2016      3193 non-null flo

## 5. Unemployment Rate

In [311]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.xls?v=43244

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1456k  100 1456k    0     0   232k      0  0:00:06  0:00:06 --:--:--  389k   0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0


In [312]:
# rename the downloaded file to education.xls
!mv Unemployment.xls?v=43244 unemployment.xls

In [313]:
# load unemployment data from Excel file
xl = pd.ExcelFile('unemployment.xls')
xl.sheet_names

['Unemployment Med HH Inc', 'Variable Descriptions']

In [314]:
# load a sheet into a pandas df by index 
unemployment = xl.parse(0, skiprows = 7)

In [315]:
# select columns that only contain relevant data for 2016
unemployment = unemployment[list(unemployment.columns[:3]) + [column for column in unemployment.columns if '2016' in column]]

In [316]:
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 9 columns):
FIPStxt                                      3275 non-null int64
State                                        3275 non-null object
Area_name                                    3275 non-null object
Civilian_labor_force_2016                    3272 non-null float64
Employed_2016                                3272 non-null float64
Unemployed_2016                              3272 non-null float64
Unemployment_rate_2016                       3272 non-null float64
Median_Household_Income_2016                 3193 non-null float64
Med_HH_Income_Percent_of_State_Total_2016    3192 non-null float64
dtypes: float64(6), int64(1), object(2)
memory usage: 230.4+ KB


In [317]:
# remove all null values from median_household_income_2016 column (3193/3275 nonnull values)
unemployment_no_na = unemployment.loc[unemployment.Median_Household_Income_2016.notnull(), :]

In [318]:
unemployment_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3195
Data columns (total 9 columns):
FIPStxt                                      3193 non-null int64
State                                        3193 non-null object
Area_name                                    3193 non-null object
Civilian_labor_force_2016                    3193 non-null float64
Employed_2016                                3193 non-null float64
Unemployed_2016                              3193 non-null float64
Unemployment_rate_2016                       3193 non-null float64
Median_Household_Income_2016                 3193 non-null float64
Med_HH_Income_Percent_of_State_Total_2016    3192 non-null float64
dtypes: float64(6), int64(1), object(2)
memory usage: 249.5+ KB


# 6. Drug Overdose Death Rate

Drug overdose death data can be downloaded from https://wonder.cdc.gov/controller/saved/D77/D37F955. From here, I agree to the CDC terms and generate a dataset with the preset search query information that is already preloaded. Then, I export the data file `Drug Overdose Deaths in 2016.txt` and save it into my working directory. I also added this file to my github account so that it's readily accessible for anyone to access.

In [319]:
# download data using bash command curl
!curl -O https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/Drug%20Overdose%20Deaths%20in%202016.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  173k  100  173k    0     0   119k      0  0:00:01  0:00:01 --:--:--  119k


In [320]:
cat drug_overdose.txt | wc -l

cat: drug_overdose.txt: No such file or directory
0


In [321]:
# remove the comments and notes from the txt file and select only the data. Then save the file into drug_overdose.txt
!sed -n 2,3148p 'Drug%20Overdose%20Deaths%20in%202016.txt' > drug_overdose.txt

In [322]:
# load drug overdose death data into pandas df
death_rates = pd.read_csv('drug_overdose.txt', 
                             delimiter = r"[\t]", 
                             engine = 'python', 
                             header = None, 
                             na_values=['Suppressed', 'Missing', 'Unreliable'])

# insert column names into df
death_rates.columns = ['county','fips_code', 'deaths', 'population', 'crude_rate']

# remove double quotes from data
death_rates.county = death_rates.county.str.replace('"', "")
death_rates.fips_code = death_rates.fips_code.str.replace('"', "")

# convert appropriate columns into numeric data type
death_rates.fips_code = pd.to_numeric(death_rates.fips_code)
death_rates.population = pd.to_numeric(death_rates.population)

# drop na values for crude_rate column
death_rates_no_na = death_rates[death_rates.crude_rate.notnull()]

death_rates_no_na.head()

,county,fips_code,deaths,population,crude_rate
1,"Baldwin County, AL",1003,32.0,208563.0,15.3
36,"Jefferson County, AL",1073,200.0,659521.0,30.3
44,"Madison County, AL",1089,61.0,356967.0,17.1
48,"Mobile County, AL",1097,64.0,414836.0,15.4
57,"St. Clair County, AL",1115,29.0,88019.0,32.9


In [323]:
death_rates_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 1 to 3122
Data columns (total 5 columns):
county        579 non-null object
fips_code     579 non-null int64
deaths        579 non-null float64
population    579 non-null float64
crude_rate    579 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 27.1+ KB


## 7. Merge the Six Datasets Together

In [324]:
# merge the six data sets together by county code (FIPS code) and drop redundant columns
death_prescribe = pd.merge(death_rates_no_na, prescription_rates_no_na, left_on = 'fips_code', right_on = 'county_code').drop(['county_x', 'county_code'], axis = 1)

dpp = pd.merge(death_prescribe, poverty_no_na, left_on = 'fips_code', right_on = 'FIPStxt').drop(['FIPStxt', 'Area_Name'], axis = 1)

dppp = pd.merge(dpp, pop_2016_no_na, left_on = 'fips_code', right_on = 'FIPS').drop(['State_x', 'FIPS', 'State_y', 'Area_Name'], axis = 1)

dpppe = pd.merge(dppp, education_no_na, left_on = 'fips_code', right_on = 'FIPS Code').drop(['FIPS Code', 'State', 'Area name'], axis = 1)

merged_data = pd.merge(dpppe, unemployment_no_na, left_on = 'fips_code', right_on = 'FIPStxt').drop(['FIPStxt', 'State', 'Area_name'], axis = 1)

merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 0 to 578
Data columns (total 57 columns):
fips_code                                                                     579 non-null int64
deaths                                                                        579 non-null float64
population                                                                    579 non-null float64
crude_rate                                                                    579 non-null float64
county_y                                                                      579 non-null object
state                                                                         579 non-null object
prescribe_rate                                                                579 non-null float64
POVALL_2016                                                                   579 non-null float64
CI90LBAll_2016                                                                579 non-null float64
CI90UBAL

In [325]:
merged_data.shape

(579, 57)